In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, RobustScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import f_regression, SelectKBest, VarianceThreshold, SelectKBest
import statsmodels.api as sm
from pandas import to_datetime
from pycaret.regression import *
from datetime import datetime
from test_functions import *
from EDA_and_preprocessing.functions import *

In [50]:
df = pd.read_csv('EDA_and_preprocessing/G_PSQL_data/final_flight.csv')
test = pd.read_csv('EDA_and_preprocessing/G_PSQL_data/final_flight_test.csv')

In [60]:
df.columns

Index(['fl_date', 'mkt_carrier_fl_num', 'origin_city_name', 'dest_city_name',
       'dest_city_name.1', 'origin', 'op_unique_carrier', 'tail_num',
       'origin_airport_id', 'dest_airport_id', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'taxi_in', 'crs_arr_time', 'arr_time',
       'arr_delay', 'crs_elapsed_time', 'actual_elapsed_time', 'dep_hour',
       'arr_hour', 'weekday', 'month_day', 'dest_weather',
       'airline_delay_category', 'total_flights', 'flight_bins', 'late_binary',
       'reduced_delay_flight', 'airport_weekly_hourly_arr',
       'airport_weekly_hourly_dep', 'airport_arr_hourly_late_percent',
       'airport_dep_hourly_late_percent', 'airport_taxi_in',
       'airport_taxi_out', 'taxi_total', 'tail_arr', 'tail_dep', 'tail_taxi',
       'tail_precent_late', 'tail_percent_reduce_delay', 'carrier_arr',
       'carrier_dep', 'carrier_percent_late', 'carrier_taxi_total',
       'percent_carrier_delay_reduced', 'arr_hourly_arr', 'arr_hourly_dep',
      

In [56]:
df['dist_times'] = pd.qcut(df['crs_elapsed_time'], 3, labels=['short', 'medium', 'long'])

In [57]:
tail = df.groupby(['tail_num', 'dist_times']).agg(tail_dist_dep=("dep_delay", 'mean'), tail_dist_arr=('arr_delay', 'mean')).reset_index()
df = df.merge(tail, on=['tail_num', 'dist_times'], how='left')
test = test.merge(tail, on=['tail_num', 'dist_times'], how='left')

In [11]:
ad = df.groupby('op_unique_carrier', as_index=False).agg({'airline_delay_category': 'mean'})
test = test.merge(ad, on='op_unique_carrier', how='left')

In [65]:
df

,fl_date,mkt_carrier_fl_num,origin_city_name,dest_city_name,dest_city_name.1,origin,op_unique_carrier,tail_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,crs_elapsed_time,actual_elapsed_time,dep_hour,arr_hour,weekday,month_day,dest_weather,airline_delay_category,total_flights,flight_bins,late_binary,reduced_delay_flight,airport_weekly_hourly_arr,airport_weekly_hourly_dep,airport_arr_hourly_late_percent,airport_dep_hourly_late_percent,airport_taxi_in,airport_taxi_out,taxi_total,tail_arr,tail_dep,tail_taxi,tail_precent_late,tail_percent_reduce_delay,carrier_arr,carrier_dep,carrier_percent_late,carrier_taxi_total,percent_carrier_delay_reduced,arr_hourly_arr,arr_hourly_dep,arr_hour_taxi,arr_hourly_percent_late,arr_hourly_percent_reduced_delay,dep_hourly_arr,dep_hourly_dep,dep_hourly_taxi,dep_hourly_percent_late,dep_hourly_percent_reduced,weekday_arr,weekday_dep,weekday_taxi,weekly_percent_late,weekly_reduced_delay,route_arr_mean,route_dep_mean,carrier_route_arr_mean,carrier_route_dep_mean,carrier_route_air_percent,tail_route_arr_mean,tail_route_dep_mean,tail_route_air_ratio,dist_times,tail_dist_dep,tail_dist_arr
0,2019-12-15,1586,"Greer, SC","Dallas/Fort Worth, TX","Dallas/Fort Worth, TX",GSP,AA,N938NN,11996,11298,738,737.0,-1.0,21.0,15.0,928,940.0,12.0,170,183.0,8.0,9.0,6,15,1.0,3,2341,5,1,0,3.836364,-5.583333,34.685315,42.647059,12.175807,16.606727,36.0,-1.415385,5.061538,27.600000,33.846154,72.307692,-1.604797,4.976720,35.012380,27.737607,72.406176,-4.029615,1.040091,28.430018,30.032737,68.758499,-3.651808,2.036207,28.338372,31.164340,69.804876,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,12.0,-1.0,107.647059,long,6.820513,-0.307692
1,2019-12-15,1612,"Greer, SC","Dallas/Fort Worth, TX","Dallas/Fort Worth, TX",GSP,AA,N932NN,11996,11298,1550,1546.0,-4.0,11.0,14.0,1731,1723.0,-8.0,161,157.0,16.0,18.0,6,15,1.0,3,2341,5,0,1,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,25.0,-5.514706,1.426471,27.794118,30.882353,79.411765,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,-8.0,-4.0,97.515528,long,-0.346154,-9.384615
2,2019-12-22,1612,"Greer, SC","Dallas/Fort Worth, TX","Dallas/Fort Worth, TX",GSP,AA,N814AW,11996,11298,1547,1703.0,76.0,11.0,13.0,1733,1822.0,49.0,166,139.0,16.0,18.0,6,22,1.0,3,2341,5,1,1,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,24.0,8.843750,14.796875,26.421875,50.000000,68.750000,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,49.0,76.0,83.734940,long,27.888889,11.777778
3,2019-12-29,1612,"Greer, SC","Dallas/Fort Worth, TX","Dallas/Fort Worth, TX",GSP,AA,N804AW,11996,11298,1547,1540.0,-7.0,14.0,8.0,1733,1732.0,-1.0,166,172.0,16.0,18.0,6,29,1.0,3,2341,5,0,0,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,22.0,0.000000,1.961538,26.948718,44.871795,62.820513,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,-1.0,-7.0,103.614458,long,4.315789,-0.631579
4,2019-12-16,1586,"Greer, SC","Dallas/Fort Worth, TX","Dallas/Fort Worth, TX",GSP,AA,N951AA,11996,11298,738,736.0,-2.0,10.0,15.0,928,924.0,-4.0,170,168.0,8.0,9.0,0,16,1.0,3,2341,5,0,1,-2.367925,-0.615385,34.685315,42.647059,12.175807,16.606727,25.0,3.852941,8.382353,27.029412,42.647059,73.529412,-1.604797,4.976720,35.012380,27.737607,72.406176,-4.029615,1.040091,28.430018,30.032737,68.758499,-3.651808,2.036207,28.338372,31.164340,69.804876,1.135630,6.673468,26.158

In [69]:
X_test.to_csv('EDA_and_preprocessing/G_PSQL_data/X_test.csv', index=False)
X.to_csv('EDA_and_preprocessing/G_PSQL_data/X_train.csv', index=False)

In [52]:
from EDA_and_preprocessing.functions import *
df = remove_outliers(df)
df = process_binary(df)
df = airport_hour_avgs(df)
df = tail_num_avg(df)
df = process_carriers(df)
df = process_hourly(df)
df = process_weekly(df)
df = process_routes(df)

In [53]:
from test_functions import *
test = process_times2(test)
test = combine_tail_num_avg(df, test)
test = combine_carriers(df, test)
test = combine_weekly(df, test)
test = combine_hourly(df, test)
test = combine_routes(df, test)
test = airport_combine_test(df, test)

In [59]:
set(test['airline_delay_category'])

{0.0, 1.0, 2.0, 3.0}

In [67]:
X = df.drop(columns=['fl_date', 'op_unique_carrier', 'tail_route_arr_mean', 'tail_route_dep_mean', 'tail_route_air_ratio', 'tail_num', 'origin_airport_id','dest_airport_id', 'crs_dep_time', 'dep_time', 'dep_delay', 'taxi_out','taxi_in', 'arr_time', 'actual_elapsed_time','weekday', 'month_day','late_binary', 'reduced_delay_flight','taxi_total','origin_city_name', 'dest_city_name', 'dest_city_name.1', 'origin','crs_arr_time',])
#X = X.drop(columns='airline_delay_category')

In [23]:
X

,arr_delay,crs_elapsed_time,dep_hour,arr_hour,dest_weather,flight_bins,airport_weekly_hourly_arr,airport_weekly_hourly_dep,airport_arr_hourly_late_percent,airport_dep_hourly_late_percent,airport_taxi_in,airport_taxi_out,tail_arr,tail_dep,tail_taxi,tail_precent_late,tail_percent_reduce_delay,carrier_arr,carrier_dep,carrier_percent_late,carrier_taxi_total,percent_carrier_delay_reduced,arr_hourly_arr,arr_hourly_dep,arr_hour_taxi,arr_hourly_percent_late,arr_hourly_percent_reduced_delay,dep_hourly_arr,dep_hourly_dep,dep_hourly_taxi,dep_hourly_percent_late,dep_hourly_percent_reduced,weekday_arr,weekday_dep,weekday_taxi,weekly_percent_late,weekly_reduced_delay,route_arr_mean,route_dep_mean,carrier_route_arr_mean,carrier_route_dep_mean,carrier_route_air_percent,tail_route_arr_mean,tail_route_dep_mean,tail_route_air_ratio
0,12.0,170.0,8.0,9.0,1.0,2,3.836364,-5.583333,34.685315,42.647059,12.175807,16.606727,-1.415385,5.061538,27.600000,33.846154,72.307692,-1.604797,4.976720,35.012380,27.737607,72.406176,-4.029615,1.040091,28.430018,30.032737,68.758499,-3.651808,2.036207,28.338372,31.164340,69.804876,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,12.0,-1.0,107.647059
1,-8.0,161.0,16.0,18.0,1.0,2,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,-5.514706,1.426471,27.794118,30.882353,79.411765,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,-8.0,-4.0,97.515528
2,49.0,166.0,16.0,18.0,1.0,2,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,8.843750,14.796875,26.421875,50.000000,68.750000,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,49.0,76.0,83.734940
3,-1.0,166.0,16.0,18.0,1.0,2,3.126531,22.363636,40.669145,44.444444,12.175807,16.606727,0.000000,1.961538,26.948718,44.871795,62.820513,-1.604797,4.976720,35.012380,27.737607,72.406176,0.880713,7.621671,24.567071,37.668919,74.160374,0.387179,7.561879,24.309014,37.714232,74.656756,-0.439255,6.269405,25.143347,36.386994,73.425852,-1.0,7.878049,9.526316,14.947368,96.736375,-1.0,-7.0,103.614458
4,-4.0,170.0,8.0,9.0,1.0,2,-2.367925,-0.615385,34.685315,42.647059,12.175807,16.606727,3.852941,8.382353,27.029412,42.647059,73.529412,-1.604797,4.976720,35.012380,27.737607,72.406176,-4.029615,1.040091,28.430018,30.032737,68.758499,-3.651808,2.036207,28.338372,31.164340,69.804876,1.135630,6.673468,26.158460,39.272378,70.252257,-1.0,7.878049,9.526316,14.947368,96.736375,-4.0,-2.0,98.823529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345117,-14.0,170.0,11.0,14.0,2.0,2,-6.684211,-1.900000,32.283465,33.783784,5.649908,15.795300,3.091837,12.234694,16.020408,35.714286,81.632653,2.813413,11.057858,42.185849,17.595968,81.066260,-1.723694,5.404239,24.438367,33.678457,75.097760,-2.528805,5.158672,24.986611,32.252603,76.504386,-0.195599,7.079331,25.251687,36.487894,74.670073,-14.0,-5.000000,-14.000000,-5.000000,94.705882,-14.0,-5.0,94.705882
345118,51.0,130.0,20.0,21.0,2.0,2,10.777778,-2.352941,50.806452,33.043478,6.430598,13.982505,2.823529,11.105882,18.188235,41.176471,77.647059,2.813413,11.057858,42.185849,17.595968,81.066260,1.708331,8.922194,25.274945,40.112180,74.702825,2.232210,10.057042,25.073464,40.517727,76.339080,-0.195599,7.079331,25.251687,36.487894,74.670073,19.0,36.000000,19.000000,36.000000,86.923077,51.0,70.0,85.384615
345119,-13.0,130.0,20.0,21.0,2.0,2,10.777778,-2.352941,50.806452,33.043478,6.430598,13.982505,1.481013,10.696203,17.443038,43.037975,78.481013,2.813413,11.057858,42.185849,17.595968,81.066260,1.708331,8.922194,25

In [68]:
X_test = test
cols = X.drop(columns='arr_delay').columns
cols = cols
X_test = X_test[cols]

In [24]:
exp1 = setup(data=X,target='arr_delay', ignore_low_variance=False, combine_rare_levels=True, feature_interaction=False, feature_selection=False, feature_selection_threshold=0.6, feature_ratio=False, polynomial_features=False, remove_outliers=True, remove_multicollinearity=True, transformation=True, fold=4)

,Description,Value
0,session_id,6261
1,Target,arr_delay
2,Original Data,"(345122, 45)"
3,Missing Values,True
4,Numeric Features,42
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(229505, 42)"


In [25]:
light1 = create_model('lightgbm', fold=2)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,5.7304,118.1769,10.8709,0.7762,0.6827,0.6361
1,5.7128,115.8791,10.7647,0.7787,0.6829,0.6341
Mean,5.7216,117.0280,10.8178,0.7775,0.6828,0.6351
Std,0.0088,1.1489,0.0531,0.0012,0.0001,0.0010


In [ ]:
best = compare_models()

IntProgress(value=0, description='Processing: ', max=99)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,01:22:56
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,8.0986,153.3977,12.3849,0.7083,0.7544,0.9014,1.5725
ridge,Ridge Regression,8.0986,153.3977,12.3849,0.7083,0.7544,0.9014,0.1725
br,Bayesian Ridge,8.0984,153.3977,12.3849,0.7083,0.7544,0.9013,1.0875
lar,Least Angle Regression,8.1086,153.4959,12.3889,0.7081,0.7548,0.9028,0.1700
omp,Orthogonal Matching Pursuit,8.2245,156.7402,12.5192,0.7020,0.7627,0.9233,0.1900
lasso,Lasso Regression,7.9795,158.5587,12.5914,0.6985,0.7547,0.8668,0.2225
huber,Huber Regressor,7.7543,160.4203,12.6651,0.6950,0.7352,0.7629,7.7700
en,Elastic Net,8.8465,186.8621,13.6693,0.6447,0.8172,0.8221,0.1950
par,Passive Aggressive Regressor,11.0292,229.5869,15.1443,0.5634,0.9178,1.3080,0.6675
llar,Lasso Least Angle Regression,16.6342,525.8800,22.9320,-0.0000,1.8897,0.9683,0.2175


In [27]:
best = load_model('best_model_tuned')

Transformation Pipeline and Model Successfully Loaded
